In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch  
import torch.nn as nn  
import torch.optim as optim   
import torch.nn.functional as F

import numpy as np
import random
import pandas as pd

# Load Data

In [ ]:
front = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/move_file/model/front_label.csv')
left = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/move_file/model/left_label.csv')
right = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/move_file/model/right_label.csv')

In [ ]:
front_data = front[['Anchor','Compare','label']]
left_data = left[['Anchor','Compare','label']]
right_data = right[['Anchor','Compare','label']]

In [ ]:
label_front = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/front_all_label.csv')['mode_trim_round']
label_left = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/left_all_label.csv')['mode_trim_round']
label_right = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/data/트리계열필요데이터/right_all_label.csv')['mode_trim_round']

In [ ]:
label_left.head()

,Anchor,Compare,Anchor_ratio,Compare_ratio,0,1,2,3,4,5,...,cosine,euclidean,Anchor.1,Compare.1,avg,mode_trim_label,only_trim_label,label,mode_trim_round,only_trim_round
0,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190923...,[0.16666667 0.50735294 0.21691176 0.24264706 0...,[0.22794118 0.53543307 0.21259843 0.24409449 0...,0.061275,0.028080,0.004313,0.001447,0.005311,0.008333,...,0.455053,9.240683,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190923...,2.6,3.0,2.666667,3,3,3
1,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/191004...,[0.16666667 0.50735294 0.21691176 0.24264706 0...,[0.16666667 0.52816901 0.21126761 0.25352113 0...,0.000000,0.020816,0.005644,0.010874,0.016071,0.032613,...,0.499617,9.421427,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/191004...,2.6,2.0,2.333333,2,2,2
2,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190802...,[0.19858156 0.50357143 0.20714286 0.23571429 0...,[0.20610687 0.50384615 0.22307692 0.21538462 0...,0.007525,0.000275,0.015934,0.020330,0.021811,0.039033,...,0.353754,8.639903,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190802...,2.8,4.0,3.000000,4,4,3
3,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190807...,[0.19858156 0.50357143 0.20714286 0.23571429 0...,[0.225 0.58394161 0.21532847 0.25547445 0...,0.026418,0.080370,0.008186,0.019760,0.020490,0.001921,...,0.357342,8.170934,/content/drive/Shareddrives/기업연계6조/data/190730...,/content/drive/Shareddrives/기업연계6조/data/190807...,2.0,2.0,2.000000,2,2,2
4,/content/drive/Shareddrives/기업연계6조/data/190813...,/content/drive/Shareddrives/기업연계6조/data/190802...,[0.23333333 0.57692308 0.20833333 0.25 0...,[0.20930233 0.50390625 0.23046875 0.2265625 0...,0.024031,0.073017,0.022135,0.023438,0.012683,0.003599,...,0.627007,11.055879,/content/drive/Shareddrives/기업연계6조/data/190813...,/content/drive/Shareddrives/기업연계6조/data/190802...,2.6,2.0,2.333333,2,2,2


In [ ]:
def load_data(file_name):
  import pickle
  import os

  save_path = '/content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data'
  os.chdir(save_path)

  with open(file_name, 'rb') as fp:
    print(f'now your current directory is {save_path}')
    return pickle.load(fp)

In [ ]:
se_t = pd.read_csv('/content/drive/Shareddrives/기업연계 NEW_6/testset.csv') 

In [ ]:
se_t.head()

,Unnamed: 0,file_list,img_representation_list,ratio_list
0,0,/content/drive/Shareddrives/기업연계 NEW_6/3...,[ 1.29320014e+00 -1.38880789e+00 7.93579891e-...,[0.25837321 0.44092827 0.21097046 0.17299578 0...
1,1,/content/drive/Shareddrives/기업연계 NEW_6/3...,[ 9.33992863e-01 -1.24328029e+00 2.98663378e-...,[0.19900498 0.56460674 0.24719101 0.24719101 0...
2,2,/content/drive/Shareddrives/기업연계 NEW_6/3...,[ 0.51588225 -1.6503826 -0.22047721 -1.104358...,[0.43373494 0.27302632 0.15789474 0.07894737 0...
3,3,/content/drive/Shareddrives/기업연계 NEW_6/3...,[ 1.4914857 -0.41811192 -0.5871361 -1.697646...,[0.1875 0.45283019 0.21293801 0.17520216 0...
4,4,/content/drive/Shareddrives/기업연계 NEW_6/3...,[ 9.31912661e-01 -1.46964324e+00 -5.63905761e-...,[0.15644172 0.59489051 0.25547445 0.25547445 0...


In [ ]:
for img1 in se_t:
  for img2 in se_t:
    img1_emb = img1['img_representation_list']
    img2_emb = img2['img_representation_list']
    img_


In [ ]:
embed_1 = se_t['img_representation_list']

'[ 1.29320014e+00 -1.38880789e+00  7.93579891e-02 -1.35911286e+00\n -2.68589519e-02 -1.80903351e+00  4.44208264e-01 -5.37762165e-01\n -7.61979282e-01 -1.35237265e+00  1.37692964e+00 -1.87007618e+00\n -9.81842518e-01 -4.99187782e-02  3.28715742e-01 -9.34916437e-01\n  1.41877401e+00 -3.41642857e-01  4.07261699e-01 -9.08754826e-01\n -2.40163946e+00 -4.48813558e-01 -1.13925278e+00 -5.23833573e-01\n  5.66815846e-02  6.79660320e-01  1.91114116e+00  2.51962900e-01\n -7.52375722e-01  6.61146641e-01 -5.09378389e-02  1.80182528e+00\n -2.24188581e-01 -3.14784408e-01  3.50450426e-01  1.30209315e+00\n -3.58574420e-01  1.61454141e-01  4.61094230e-01  2.23850489e-01\n  1.38512266e+00 -5.11399686e-01  1.38506427e-01  8.22111815e-02\n  3.33192557e-01 -2.34042096e+00  9.01010692e-01 -7.66560197e-01\n -1.18993565e-01  5.22555828e-01  2.92034894e-02 -6.69375509e-02\n  8.96290183e-01 -1.52918494e+00 -4.28616941e-01  7.64409721e-01\n  2.62755930e-01  1.10703075e+00  8.11255276e-02 -7.88273990e-01\n  3.66306

In [ ]:
for 

## Load Embedding & Ratio

In [ ]:
## img1,2 for Embedding, ratio1,2 for Ratio

img1_train = load_data('img1_train')
img2_train = load_data('img2_train')
ratio1_train = load_data('ratio1_train')
ratio2_train = load_data('ratio2_train')

img1_left = load_data('left_img1_train')
img2_left = load_data('left_img2_train')
ratio1_left = load_data('left_ratio1_train')
ratio2_left = load_data('left_ratio2_train')

img1_right = load_data('right_img1_train')
img2_right = load_data('right_img2_train')
ratio1_right = load_data('right_ratio1_train')
ratio2_right = load_data('right_ratio2_train')

now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/drive/Shareddrives/기업연계 NEW_6/move_file/model/input_data
now your current directory is /content/d

In [ ]:
print(img1_train.shape)
print(img2_train.shape)
print(ratio1_train.shape)
print(ratio2_train.shape)
print(img1_left.shape)
print(ratio1_left.shape)
print(img1_right.shape)
print(ratio1_right.shape)

(2000, 128)
(2000, 128)
(2000, 18)
(2000, 18)
(800, 128)
(800, 17)
(800, 128)
(800, 17)


In [ ]:
'''
## Set Labels of Dataset

y = front_data['label']
y_left = left_data['label']
y_right = right_data['label']
'''
y = label_front
y_left = label_left
y_right = label_right

In [ ]:
print(y.shape,y_left.shape,y_right.shape)

(2000,) (800,) (800,)


## Calculate Distance

In [ ]:
# l1
l1= np.sum(np.abs(img1_train,img2_train),axis=1)
l1_left = np.sum(np.abs(img1_left,img2_left),axis=1)
l1_right = np.sum(np.abs(img1_right,img2_right),axis=1)
# l2
l2 = np.sum((img1_train-img2_train)**2,axis=1)
l2_left = np.sum((img1_left-img2_left)**2,axis=1)
l2_right = np.sum((img1_right-img2_right)**2,axis=1)

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# cos
cos = []
cos_left = []
cos_right = []
i = 0
for row in img1_train:
  cos.append(cos_sim(row,img2_train[i]))
cos = np.array(cos)

for row in img1_left:
  cos_left.append(cos_sim(row,img2_left[i]))
cos_left = np.array(cos_left)

for row in img1_right:
  cos_right.append(cos_sim(row,img2_right[i]))
cos_right = np.array(cos_right)

In [ ]:
l1 = l1.reshape(2000,1)
l1_left = l1_left.reshape(800,1)
l1_right = l1_right.reshape(800,1)

l2 = l2.reshape(2000,1)
l2_left = l2_left.reshape(800,1)
l2_right = l2_right.reshape(800,1)

cos = cos.reshape(2000,1)
cos_left = cos_left.reshape(800,1)
cos_right = cos_right.reshape(800,1)

## Dataset build and Split

In [ ]:
dis = np.hstack((l1,l2,cos))
ratio = np.abs(ratio1_train-ratio2_train) 

dis_left = np.hstack((l1_left,l2_left,cos_left))
ratio_left = np.abs(ratio1_left-ratio2_left)

dis_right = np.hstack((l1_right,l2_right,cos_right))
ratio_right = np.abs(ratio1_right-ratio2_right)

In [ ]:
def train_test_split(ratio,distance,label,train_num):
  #np.random.seed(10)
  train_idx = random.sample(range(0,ratio.shape[0]),train_num)
  # Split Dataset
  ratio_train = ratio[train_idx]
  ratio_test = np.delete(ratio,train_idx,axis=0)
  distance_train = distance[train_idx]
  distance_test = np.delete(distance,train_idx,axis=0)
  y_train = label[train_idx]
  y_test = np.delete(label,train_idx,axis=0)
  # Allocate each set
  training_set = torch.FloatTensor(ratio_train),torch.FloatTensor(distance_train),torch.LongTensor(y_train)
  test_set = torch.FloatTensor(ratio_test),torch.FloatTensor(distance_test),torch.LongTensor(y_test)

  return training_set, test_set

In [ ]:
trainset,testset = train_test_split(ratio,dis,np.array(y),1800)
train_left,test_left = train_test_split(ratio_left,dis_left,np.array(y_left),600)
train_right,test_right = train_test_split(ratio_right,dis_right,np.array(y_right),600)

In [ ]:
ratio_train = trainset[0]
distance_train = trainset[1]
y_train = trainset[2]

ratio_left_train = train_left[0]
distance_left_train = train_left[1]
y_left_train = train_left[2]

ratio_right_train = train_right[0]
distance_right_train = train_right[1]
y_right_train = train_right[2]

print('Training Set')
print(ratio_train.shape)
print(distance_train.shape)
print(y_train.shape)
print()

ratio_test = testset[0]
distance_test = testset[1]
y_test = testset[2]

ratio_left_test = test_left[0]
distance_left_test = test_left[1]
y_left_test = test_left[2]

ratio_right_test = test_right[0]
distance_right_test = test_right[1]
y_right_test = test_right[2]

print('Test Set')
print(ratio_test.shape)
print(distance_test.shape)
print(y_test.shape)
print()

Training Set
torch.Size([1800, 18])
torch.Size([1800, 3])
torch.Size([1800])

Test Set
torch.Size([200, 18])
torch.Size([200, 3])
torch.Size([200])



# Model Build

In [ ]:
## Model Build

# First, inherit the base module from torch.nn.Module
class Predict(nn.Module):

  def __init__(self,ratio_num): # ratio_num is number of our ratio we use
    super().__init__()  # take all info of __init__ from nn.Module

    self.ratio_num = ratio_num

    ## FC Layer for extract feature vector of ratio
    self.FC1 = nn.Linear(ratio_num,32)
    nn.init.xavier_normal(self.FC1.weight)

    self.FC2 = nn.Linear(32,20)
    nn.init.xavier_normal(self.FC2.weight)

    self.FC3 = nn.Linear(20,15)
    nn.init.xavier_normal(self.FC3.weight)
    # We set the number of nodes at last fully connected layer same with 'ratio_number - number of randomly extracted ratio'

    ## FC Layer for reconsider some ratio
    self.FC4 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC4.weight)

    self.FC5 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC5.weight)

    self.FC6 = nn.Linear(18,9)
    nn.init.xavier_normal(self.FC6.weight)

    ## FC Layer for consider latent of ratio and distance btw embedding vectors

    self.FC7 = nn.Linear(12,8)
    nn.init.xavier_normal(self.FC7.weight)

    self.FC8 = nn.Linear(8,6) # Penultimate Layer of this model
    nn.init.xavier_normal(self.FC8.weight)
    
    ## Output Layer
    self.Output = nn.Linear(6,5)

    ## Define Activation Function
    self.ReLU = nn.ReLU()
    self.Softmax = nn.Softmax()
    self.Tanh = nn.Tanh()
    self.LeakR = nn.LeakyReLU()

    ## Layer for Regularization
    self.dropout = nn.Dropout(0.25)
    
  def forward(self,ratio_tensor_array,distance):

    # Load Distance of Embedding
    self.distance_tensor = distance

    ## Forward Pass for Extract Ratio Feature

    ratio_tensor_array = F.normalize(ratio_tensor_array)  # Normalize Input Ratio
    
    y1 = self.FC1(ratio_tensor_array)
    a1 = self.ReLU(y1)
    a1 = F.normalize(a1)

    y2 = self.FC2(a1)
    a2 = self.ReLU(y2)
    a2 = F.normalize(a2)

    y3 = self.FC3(a2)
    a3 = self.ReLU(y3)
    a3 = F.normalize(a3)

    ## Forward Pass for Reminder Part 

    # Extract with Replacement

    random_ratio_idx = np.random.randint(0,self.ratio_num,size=3)
    remind_y1 = self.FC4(torch.cat( [a3, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a1 = self.ReLU(remind_y1)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y2 = self.FC5(torch.cat( [remind_a1, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a2 = self.ReLU(remind_y2)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y3 = self.FC6(torch.cat( [remind_y2, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a3 = self.ReLU(remind_y3)


    ## Forward Pass for Fully Considered Part

    whole_y1 = self.FC7( torch.cat( [remind_a3,self.distance_tensor],dim=1 ))
    whole_a1 = self.ReLU(whole_y1)

    whole_y2 = self.FC8(whole_a1)
    whole_a2 = self.ReLU(whole_y2)

    return self.Output(whole_a2)

In [ ]:
## Model Build

# First, inherit the base module from torch.nn.Module
class Predict_left(nn.Module):

  def __init__(self,ratio_num): # ratio_num is number of our ratio we use
    super().__init__()  # take all info of __init__ from nn.Module

    self.ratio_num = ratio_num

    ## FC Layer for extract feature vector of ratio
    self.FC1 = nn.Linear(ratio_num,32)
    nn.init.xavier_normal(self.FC1.weight)

    self.FC2 = nn.Linear(32,20)
    nn.init.xavier_normal(self.FC2.weight)

    self.FC3 = nn.Linear(20,15)
    nn.init.xavier_normal(self.FC3.weight)
    # We set the number of nodes at last fully connected layer same with 'ratio_number - number of randomly extracted ratio'

    ## FC Layer for reconsider some ratio
    self.FC4 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC4.weight)

    self.FC5 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC5.weight)

    self.FC6 = nn.Linear(18,9)
    nn.init.xavier_normal(self.FC6.weight)

    ## FC Layer for consider latent of ratio and distance btw embedding vectors

    self.FC7 = nn.Linear(12,8)
    nn.init.xavier_normal(self.FC7.weight)

    self.FC8 = nn.Linear(8,6) # Penultimate Layer of this model
    nn.init.xavier_normal(self.FC8.weight)
    
    ## Output Layer
    self.Output = nn.Linear(6,5)

    ## Define Activation Function
    self.ReLU = nn.ReLU()
    self.Softmax = nn.Softmax()
    self.Tanh = nn.Tanh()
    self.LeakR = nn.LeakyReLU()

    ## Layer for Regularization
    self.dropout = nn.Dropout(0.25)
    
  def forward(self,ratio_tensor_array,distance):

    # Load Distance of Embedding
    self.distance_tensor = distance

    ## Forward Pass for Extract Ratio Feature

    ratio_tensor_array = F.normalize(ratio_tensor_array)  # Normalize Input Ratio
    
    y1 = self.FC1(ratio_tensor_array)
    a1 = self.ReLU(y1)
    a1 = F.normalize(a1)

    y2 = self.FC2(a1)
    a2 = self.ReLU(y2)
    a2 = F.normalize(a2)

    y3 = self.FC3(a2)
    a3 = self.ReLU(y3)
    a3 = F.normalize(a3)

    ## Forward Pass for Reminder Part 

    # Extract with Replacement

    random_ratio_idx = np.random.randint(0,self.ratio_num,size=3)
    remind_y1 = self.FC4(torch.cat( [a3, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a1 = self.ReLU(remind_y1)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y2 = self.FC5(torch.cat( [remind_a1, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a2 = self.ReLU(remind_y2)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y3 = self.FC6(torch.cat( [remind_y2, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a3 = self.ReLU(remind_y3)


    ## Forward Pass for Fully Considered Part

    whole_y1 = self.FC7( torch.cat( [remind_a3,self.distance_tensor],dim=1 ))
    whole_a1 = self.ReLU(whole_y1)

    whole_y2 = self.FC8(whole_a1)
    whole_a2 = self.ReLU(whole_y2)

    return self.Output(whole_a2)

In [ ]:
## Model Build

# First, inherit the base module from torch.nn.Module
class Predict_right(nn.Module):

  def __init__(self,ratio_num): # ratio_num is number of our ratio we use
    super().__init__()  # take all info of __init__ from nn.Module

    self.ratio_num = ratio_num

    ## FC Layer for extract feature vector of ratio
    self.FC1 = nn.Linear(ratio_num,32)
    nn.init.xavier_normal(self.FC1.weight)

    self.FC2 = nn.Linear(32,20)
    nn.init.xavier_normal(self.FC2.weight)

    self.FC3 = nn.Linear(20,15)
    nn.init.xavier_normal(self.FC3.weight)
    # We set the number of nodes at last fully connected layer same with 'ratio_number - number of randomly extracted ratio'

    ## FC Layer for reconsider some ratio
    self.FC4 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC4.weight)

    self.FC5 = nn.Linear(18,15)
    nn.init.xavier_normal(self.FC5.weight)

    self.FC6 = nn.Linear(18,9)
    nn.init.xavier_normal(self.FC6.weight)

    ## FC Layer for consider latent of ratio and distance btw embedding vectors

    self.FC7 = nn.Linear(12,8)
    nn.init.xavier_normal(self.FC7.weight)

    self.FC8 = nn.Linear(8,6) # Penultimate Layer of this model
    nn.init.xavier_normal(self.FC8.weight)
    
    ## Output Layer
    self.Output = nn.Linear(6,5)

    ## Define Activation Function
    self.ReLU = nn.ReLU()
    self.Softmax = nn.Softmax()
    self.Tanh = nn.Tanh()
    self.LeakR = nn.LeakyReLU()

    ## Layer for Regularization
    self.dropout = nn.Dropout(0.25)
    
  def forward(self,ratio_tensor_array,distance):

    # Load Distance of Embedding
    self.distance_tensor = distance

    ## Forward Pass for Extract Ratio Feature

    ratio_tensor_array = F.normalize(ratio_tensor_array)  # Normalize Input Ratio
    
    y1 = self.FC1(ratio_tensor_array)
    a1 = self.ReLU(y1)
    a1 = F.normalize(a1)

    y2 = self.FC2(a1)
    a2 = self.ReLU(y2)
    a2 = F.normalize(a2)

    y3 = self.FC3(a2)
    a3 = self.ReLU(y3)
    a3 = F.normalize(a3)

    ## Forward Pass for Reminder Part 

    # Extract with Replacement

    random_ratio_idx = np.random.randint(0,self.ratio_num,size=3)
    remind_y1 = self.FC4(torch.cat( [a3, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a1 = self.ReLU(remind_y1)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y2 = self.FC5(torch.cat( [remind_a1, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a2 = self.ReLU(remind_y2)

    random_ratio_idx = np.random.randint(0,self.ratio_num,3)
    remind_y3 = self.FC6(torch.cat( [remind_y2, ratio_tensor_array[:,random_ratio_idx]], dim=1 ))
    remind_a3 = self.ReLU(remind_y3)


    ## Forward Pass for Fully Considered Part

    whole_y1 = self.FC7( torch.cat( [remind_a3,self.distance_tensor],dim=1 ))
    whole_a1 = self.ReLU(whole_y1)

    whole_y2 = self.FC8(whole_a1)
    whole_a2 = self.ReLU(whole_y2)

    return self.Output(whole_a2)

# Training Step

In [ ]:
model = Predict(18)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-

In [ ]:
model_left = Predict_left(17)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-

In [ ]:
model_right = Predict_right(17)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
/usr/local/lib/python3.7/dist-

In [ ]:
optimizer = optim.Adam(model_left.parameters(),lr=0.001,weight_decay=0.0001)

In [ ]:
# One hot encoding : Left Training
y_one_hot_left = torch.zeros(600,5)
y_one_hot_left.scatter(1, y_left_train.unsqueeze(1)-1,1)
print(y_one_hot_left.shape)

torch.Size([600, 5])


In [ ]:
# One hot encoding : Left Test
y_one_hot_left_test = torch.zeros(200,5)
y_one_hot_left_test.scatter(1, y_left_test.unsqueeze(1)-1,1)
print(y_one_hot_left_test.shape)

torch.Size([200, 5])


In [ ]:
# One hot encoding : Right Training
y_one_hot_right = torch.zeros(600,5)
y_one_hot_right.scatter(1, y_right_train.unsqueeze(1)-1,1)
print(y_one_hot_right.shape)

torch.Size([600, 5])


In [ ]:
# One hot encoding : Right Test
y_one_hot_right_test = torch.zeros(200,5)
y_one_hot_right_test.scatter(1, y_right_test.unsqueeze(1)-1,1)
print(y_one_hot_right_test.shape)

torch.Size([200, 5])


In [ ]:
# One hot encoding : Front Training
y_one_hot = torch.zeros(1800, 5)
y_one_hot.scatter_(1, y_train.unsqueeze(1)-1,1)
print(y_one_hot.shape)

torch.Size([1800, 5])


In [ ]:
# One hot encoding : Front Test
y_one_hot_test = torch.zeros(200, 5)
y_one_hot_test.scatter_(1, y_test.unsqueeze(1)-1,1)
print(y_one_hot_test.shape)

torch.Size([200, 5])


In [ ]:
y_one_hot_left_test = torch.zeros(200, 5)
y_one_hot_left_test.scatter_(1, y_left_test.unsqueeze(1)-1,1)
print(y_one_hot_left_test.shape)

torch.Size([200, 5])


In [ ]:
y_one_hot_right_test = torch.zeros(200, 5)
y_one_hot_right_test.scatter_(1, y_right_test.unsqueeze(1)-1,1)
print(y_one_hot_right_test.shape)

torch.Size([200, 5])


In [ ]:
import os

In [ ]:
save_dir = '/content/drive/Shareddrives/기업연계 NEW_6/승연/Model_checkpoint/찐막'
model_name = 'Xavier_ReLU_Adam_Layer2_18ratio_200test_decay'
ratio_n_layers = 3
total_n_layers = 2
random_n_ratio = 3
save_every = 2000

In [ ]:
epochs = 20000
correct = 0
for epoch in range(epochs+1):

  prediction = model_left(ratio_left_train,distance_left_train)
  loss = F.cross_entropy(prediction,y_one_hot_left)

  optimizer.zero_grad()
  loss.backward() # subject of training is 'loss function' 
  optimizer.step()

  _,predicted = torch.max(prediction.data,1)
  correct = (predicted+1 == y_left_train).sum().item()
  
  model.eval()
  with torch.no_grad():
    prediction_test = model_left(ratio_left_test,distance_left_test)
    valid_loss = F.cross_entropy(prediction_test,y_one_hot_left_test)
    _,predicted_test = torch.max(prediction_test.data,1)
    correct_test = (predicted_test+1 == y_test).sum().item()
  '''
  if epoch%100 == 0:
    print('Epoch {:4d}/{} | Train Loss : {:.6f} | Train Accuracy : {:.2f}%'.format(epoch,epochs,loss.item(),(correct/predicted.shape[0])*100))
                                                                                                                               
  '''
  if epoch%100 == 0:
    print('Epoch {:4d}/{} | Train Loss : {:.6f} | Train Accuracy : {:.2f}% | Valid Loss : {:.6f} | Valid Accuracy : {:.2f}%'.format(epoch,epochs,loss.item(),(correct/predicted.shape[0])*100,
                                                                                                                                valid_loss.item(),(correct_test/predicted_test.shape[0])*100))
  
  # Save Checkpoint
  if (epoch % save_every == 0):
      directory = os.path.join(save_dir, model_name, '{}-{}_{}'.format(ratio_n_layers, total_n_layers, random_n_ratio))
      if not os.path.exists(directory):
          os.makedirs(directory)
      torch.save({
                'epoch': epoch,
                'en': model.state_dict(),
                'en_opt': optimizer.state_dict(),
                'loss': loss,
            }, os.path.join(directory, '{}_{}.tar'.format(epoch, 'checkpoint')))

Epoch    0/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.303068 | Valid Accuracy : 21.00%
Epoch  100/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.297349 | Valid Accuracy : 21.00%
Epoch  200/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.294348 | Valid Accuracy : 21.00%
Epoch  300/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.300552 | Valid Accuracy : 21.00%
Epoch  400/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.298505 | Valid Accuracy : 21.00%
Epoch  500/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.296314 | Valid Accuracy : 21.00%
Epoch  600/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.303346 | Valid Accuracy : 21.00%
Epoch  700/20000 | Train Loss : -0.000000 | Train Accuracy : 24.67% | Valid Loss : 9.296903 | Valid Accuracy : 21.00%
Epoch  800/20000 | Train Loss : -0.000000 | Train Accura

KeyboardInterrupt: ignored

11400 epoch에서 가장 accuracy 높게 나옴

In [ ]:
torch.save(model,'/content/drive/Shareddrives/기업연계 NEW_6/승연/Xavier_ReLU_Adam_0.001_Layer2.pth')

In [ ]:
torch.save(model.state_dict(),'/content/drive/Shareddrives/기업연계 NEW_6/승연/Xavier_ReLU_Adam_0.001_Layer2_params.pt' )

In [ ]:
dis = np.hstack((l1,l2,cos))
ratio = np.abs(ratio1_train-ratio2_train)[:,[1,4,3,11,16,0,5,6,17]]